### Get Cloud outage data into dataframe

First import the json and define column names and there mapping to json

In [9]:
import json
import pandas as pd

COLS = {"id": "id",    
        "start": "begin",
        "end": "end",
        "severity": "severity",
        "status": "status_impact",
        "location": ("currently_affected_locations", "previously_affected_locations"),
        "service_name": "service_name",
        "number_affected_products": "affected_products"
    }

with open("../json-data/incidents.json") as file:
    incidents_json = json.load(file)

Create data frame and copy over the data

In [24]:
df = pd.DataFrame(data=[], columns=COLS.keys())

for incident in incidents_json:
    locations = {x["id"] for x in incident.get(COLS["location"][0], None) + incident.get(COLS["location"][1], None)}
    row = {
        "id": incident.get(COLS["id"], None),
        "start": incident.get(COLS["start"], None),
        "end": incident.get(COLS["end"], None),
        "severity": incident.get(COLS["severity"], None),
        "location": locations,
        "service_name": incident.get(COLS["service_name"]),
        "status": incident.get(COLS["status"], None),
        "number_affected_products": len(incident.get(COLS["number_affected_products"], []))
    }
    df.loc[len(df)] = row

Analyze the data

In [34]:
res = df[df['severity'].isin(["high"])]
print(res[['severity', 'start', 'end']])
print(res.shape)

    severity                      start                        end
3       high  2023-09-13T06:46:00+00:00  2023-09-13T10:32:00+00:00
6       high  2023-09-11T11:00:00+00:00  2023-09-11T19:25:00+00:00
29      high  2023-08-14T23:11:00+00:00  2023-08-15T18:22:00+00:00
45      high  2023-07-28T17:17:00+00:00  2023-07-28T22:52:00+00:00
144     high  2023-04-26T08:27:34+00:00  2023-04-26T09:18:36+00:00
147     high  2023-04-26T06:45:00+00:00  2023-04-26T23:07:00+00:00
148     high  2023-04-26T02:00:00+00:00  2023-04-27T00:05:00+00:00
203     high  2023-03-09T09:30:00+00:00  2023-03-09T10:24:00+00:00
209     high  2023-03-03T19:18:00+00:00  2023-03-03T20:45:00+00:00
213     high  2023-02-27T12:58:00+00:00  2023-02-27T13:11:00+00:00
276     high  2022-12-09T12:50:00+00:00  2022-12-09T15:55:00+00:00
282     high  2022-12-02T01:30:00+00:00  2022-12-03T04:50:00+00:00
283     high  2022-12-01T20:20:00+00:00  2022-12-01T21:36:00+00:00
290     high  2022-11-14T18:50:00+00:00  2022-11-14T19:38:00+0